# DIFFERENCE ARRAY (LEET 2381)

Da usare quando si vuole fare un range update, invece che aggiornare ogni elemento da index 2 a 5 ad esempio. Incremento l’elemento all’indice 2 di + tot e tolgo all’indice end +1 – tot quindi se voglio aggiungere 2 a tutti gli elementi tra 2 e 5 devo fare

Start = [1, 2, 1, 1, 3, 0, 0]

Compute gradient array: array delle somme parziali, cioè prendo solo le differenze tra elementi

Arra_diff= [1, 1, -1, 0, 2, -3, 0, 0]   

last element is dummy. Il dummy serve se voglio aggiornare un range che ha dentro anche l’ultimo index dell’array

Aggiungo il valore a index 2 e tolgo a index 5+1:

Arra_diff= [1, 1, 1, 0, 2, -5, 0, 0]

Poi per ottenere la somma finale devo fare la somma cumulativa, l’ultimo elemento dell’array sum è dummy

Tot_sum = [1, 2, 3, 3, 5, 0,  0, 0]

Final_res = [1, 2, 3, 3, 5, 0,  0] #remove last element

In [ ]:
class Solution:
    def shiftingLetters(self, s: str, shifts: List[List[int]]) -> str:
        #0 sta dov'è, -num vai indietro e +num vai inavanti
        arr_diff = [0] * (len(s)+1)
        final_shifts = []
        for shift in shifts:
            if shift[2]:
                arr_diff[shift[0]] += 1
                arr_diff[shift[1]+1] -= 1
            else:
                arr_diff[shift[0]] -= 1
                arr_diff[shift[1]+1] += 1

        shift = 0
        for i in range(len(arr_diff)-1):
            shift += arr_diff[i]
            final_shifts.append(shift)

        res = ""
        for i, el in enumerate(final_shifts):
            new_char = chr((ord(s[i]) - ord('a') + el) % 26 + ord('a'))
            res += new_char
        return res


# IMMUTABLE QUERY RANGE (LEET 303)

non serve la tree structure perchè non abbiamo updates
quindi basta utilizzare l'array della prefix sum, cioè somma cumulativa e fare somm:  arr[right] - arr[left-1]

In [ ]:
class NumArray:

    def __init__(self, nums: List[int]):
        self.sum = [0] * (len(nums)+1)

        #somma cumulativa
        for i in range(len(nums)):
            self.sum[i+1] = self.sum[i] + nums[i]

    #ho spostato gli indici perchè ho fatto la sum di len n +1 così risulta + facile fare left-1, quindi left-1 +1 = left e right +1 = right+1
    #così posso riferirmi all'array di sum facilmente
    def sumRange(self, left: int, right: int) -> int:
        return self.sum[right+1] - self.sum[left]


# Your NumArray object will be instantiated and called as such:
# obj = NumArray(nums)
# param_1 = obj.sumRange(left,right)

# PREFIX SUM & SUFFIX SUM (LEET 1769)

In questo caso farlo senza prefix e suffix sarebbe n^2. Quello che faccio con prexis sum e suffix sum è calcolarsi prima la somma da sx a dx e poi da dx a sx, il risultato finale è dato dalla somma di prefix sum e suffix sum.

In questo caso nel leet si poteva ottenere prefix sum e suffix sum tenendosi il numero di biglie incontrate fino a quel punto e aggiornandolo in caso si trovasse una nuova biglia, e lo si aggiungeva al risultato corrente in modo che il numero totale di biglie da muovere venisse aggiornato e quindi anche il numero totale di movimenti.

In [ ]:
class Solution:
    def minOperations(self, boxes: str) -> List[int]:
        n = len(boxes)
        #computo da sx a dx e da dx a sx così posso farlo in tempo n,
        #ogni volta aggiungo il numero di palle perchè per andare da una posizione ad una adiacente mi basta
        #computare il numero di palle e aggiungerle perchè devo spostare ogni volta tot palle per inserle nel box i
        l_r = [0] * n
        r_l = [0] * n

        tot_balls = 0
        tot_operations = 0
        for i in range(n):
            tot_operations += tot_balls
            l_r[i] = tot_operations
            if boxes[i] == '1':
                tot_balls += 1

        tot_balls = 0
        tot_operations = 0
        for i in reversed(range(n)):
            tot_operations += tot_balls
            r_l[i] = tot_operations
            if boxes[i] == '1':
                tot_balls += 1

        for i in range(n):
            l_r[i] += r_l[i]
        return l_r

# DP + sliding window (leet 2017)
My intuition:
2nd robot (goes second, so the function is innested) wants to maxize what is left from either top or bottom after the pass of 1st robot, so MAX(TOP, BOTTOM).
the first robot (goes first so outer function) want to minimize the result of the second robot so MIN(MAX(TOP, BOTTOM)).
So this problem is basically solved, I just need to compute the remaining top and bottom and that's it!
So how do i do that TOP: compute the sum of the first row and as you go you decrease values.
Remaining of the BOTTOM: as you go you sum what you found on the same col, bcs since you stayed on top, the top row value goes to 0 aka difference. and the value on bottom row remains for the second robot so sum to bottom value. that's it! 





Solution:
Step 1: Sum the Top Row

Calculate the total sum of the top row (grid[0]).
This represents the score available to the second robot  initially if the first robot doesn’t take any cells from the top row.
Why? The top row sum decreases as the first robot progresses.
Step 2: Traverse Columns 

Use a sliding window approach :
Deduct the value of the current column from the top row's sum (representing the first robot collecting these cells).
Add the value of the current column to the bottom row's cumulative sum (bottom).
Calculate the maximum score left for the second robot using max(top, bottom).
Track the minimum "maximum score" possible using res.
Step 3: Minimize the Maximum 

The goal is to minimize the "worst-case scenario" for the second robot, ensuring a balanced outcome.

In [ ]:
class Solution:
    def gridGame(self, grid):
        top = sum(grid[0])
        bottom = 0
        res = float('inf')
        
        for i in range(len(grid[0])):
            top -= grid[0][i]
            res = min(res, max(top, bottom))
            bottom += grid[1][i]
        
        return res

# leet 2401

bit manipulation + sliding window with mask

In [ ]:
class Solution:
    def longestNiceSubarray(self, nums: List[int]) -> int:
        s = 0
        curr_mask = 0
        max_len = 0
        
        for e in range(len(nums)):
            while (curr_mask & nums[e]) != 0:
                # Remove nums[s] from the bitmask and move the left pointer
                curr_mask ^= nums[s]
                s += 1

            # Add nums[e] to the bitmask
            curr_mask |= nums[e]

            # Update max length
            max_len = max(max_len, e - s + 1)
        
        return max_len